In [ ]:
import findspark
findspark.init()
import pyspark
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession \
    .builder \
    .appName("ECE552_FinalProject") \
    .config("spark.driver.memory", "2g") \
    .config("spark.mongodb.input.uri", "mongodb://127.0.0.1/ECE552FinalProject") \
    .config("spark.mongodb.output.uri", "mongodb://127.0.0.1/ECE552FinalProject") \
    .getOrCreate()
sc = SparkContext.getOrCreate()
spark.sparkContext._conf.getAll()

In [ ]:
sc.setJobGroup("DA_Covid", "DA_Covid")

In [ ]:
%%time
#Declaration of Pandemic to First Vaccine
pipeline = "[{ $match :{Start_Time: {$gt:ISODate('2020-03-11'), $lt:ISODate('2020-12-14')}}}]"
df = spark.read.format("mongo").option("collection", "TrafficData").option("pipeline", pipeline).load()
df.count()

In [ ]:
df.write.format("mongo").option("collection", "AccidentCountCovidPandemicToVaccine").mode("overwrite").save()

In [ ]:
pipeline = "[{ $group : { _id :{$month:{date:'$Start_Time'}}, count:{$sum:1}}}]"
df = spark.read.format("mongo").option("collection", "AccidentCountCovidPandemicToVaccine").option("pipeline", pipeline).load()
df = df.groupby("_id").sum("count")
df = df.withColumnRenamed("_id","Month").withColumnRenamed("sum(count)","AccidentCount")
print("Number of Documents: " + str(df.count()))
df.agg({'AccidentCount': 'sum'}).show()
df.show(49)

In [ ]:
%%time
#First year of Covid

pipeline = "[{ $match :{Start_Time: {$gt:ISODate('2020-03-11'), $lt:ISODate('2021-03-11')}}}]"
df = spark.read.format("mongo").option("collection", "TrafficData").option("pipeline", pipeline).load()
print("Number of Documents: " + str(df.count()))


In [ ]:
df.write.format("mongo").option("collection", "AccidentCountCovidYear1").mode("overwrite").save()

In [ ]:
pipeline = "[{ $group : { _id :{$month:{date:'$Start_Time'}}, count:{$sum:1}}}]"
df = spark.read.format("mongo").option("collection", "AccidentCountCovidYear1").option("pipeline", pipeline).load()
df = df.groupby("_id").sum("count")
df = df.withColumnRenamed("_id","Month").withColumnRenamed("sum(count)","AccidentCount")
print("Number of Documents: " + str(df.count()))
df.agg({'AccidentCount': 'sum'}).show()
df.show(49)